In [5]:
import scipy.io
mat = scipy.io.loadmat('imdb_crop/imdb.mat')

In [6]:
dob = mat['imdb'][0, 0][0]
photo_taken = mat['imdb'][0, 0][1]
full_path = mat['imdb'][0, 0][2]
gender = mat['imdb'][0, 0][3]

from datetime import datetime, timedelta
import numpy as np


train_len = 430000
test_len = dob.shape[1] - train_len

age = np.zeros_like(dob)
false_age = 0

for i in range(train_len):
    matlab_datenum = float(dob[0][i])
    try:
      python_datetime = datetime.fromordinal(int(matlab_datenum)) + timedelta(days=matlab_datenum%1) - timedelta(days = 366)
      age[0][i] = np.abs(python_datetime.year - photo_taken[0][i])
    except:
      false_age += 1
      age[0][i] = 20

print(false_age)

111


In [7]:
import os
import shutil

for i in range(0, 20):
    directory = 'E:\\Projects\\Age_prediction\\data\\train\\'+str(i)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    directory = 'E:\\Projects\\Age_prediction\\data\\test\\'+str(i)
    if not os.path.exists(directory):
        os.makedirs(directory)

In [8]:
import random
n = dob.shape[1]

index = random.sample(range(0, n), n)

for i in range(train_len):
    src = full_path[0][index[i]][0]
    des = src.split('/')
    
    directory = 'E:\\Projects\\Age_prediction\\data\\train\\'+str(int(age[0][index[i]]/5))     
    src = "E:\\Projects\\Age_prediction\\imdb_crop\\"+str(des[0]+"/"+des[1])
    des =  directory+'/'+des[1]
    
    if age[0][index[i]]<100 and age[0][index[i]] != 0:
      shutil.copy(src, des)


for i in range(train_len, train_len + test_len):
    src = full_path[0][index[i]][0]
    des = src.split('/')
    
    directory = 'E:\\Projects\\Age_prediction\\data\\test\\'+str(int(age[0][index[i]]/5))         
    src = "E:\\Projects\\Age_prediction\\imdb_crop\\"+str(des[0]+"/"+des[1])
    des =  directory+'/'+des[1]
    
    if age[0][index[i]]<100 and age[0][index[i]] != 0:
      shutil.copy(src, des)
    

# Building the model

In [1]:
import os, shutil
train_dir = 'E:\\Projects\\Age_prediction\\data\\train\\'
test_dir = 'E:\\Projects\\Age_prediction\\data\\test\\'

In [2]:
import keras

Using TensorFlow backend.


In [3]:
from keras import models, layers, losses, optimizers, metrics
from keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(rescale=1./255, 
#                                    rotation_range=15, 
#                                    width_shift_range=0.05, 
#                                    height_shift_range=0.05, 
#                                    shear_range=0.05,
#                                    zoom_range=0.05, 
#                                    horizontal_flip=True
                                  )

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(150, 150), 
    batch_size=16
)

test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(150, 150), 
    batch_size=16
)

Found 401271 images belonging to 20 classes.
Found 28648 images belonging to 20 classes.


In [4]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(150, 150, 3))


In [5]:
model = models.Sequential()

model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(20, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               4194816   
_________________________________________________________________
dense_2 (Dense)              (None, 20)                10260     
Total params: 18,919,764
Trainable params: 18,919,764
Non-trainable params: 0
_________________________________________________________________


In [ ]:
conv_base.trainable = False
print('This is the number of trainable weights before freezing the conv base:', len(model.trainable_weights))

model.compile(
    optimizer=optimizers.RMSprop(lr=1e-4), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy]
)

history = model.fit_generator(train_generator, 
                              steps_per_epoch=25000, 
                              epochs=10, 
                              validation_data=test_generator, 
                              validation_steps=1800)

This is the number of trainable weights before freezing the conv base: 30
Epoch 1/10
25000/25000 [==============================] - 6470s - loss: 2.2340 - categorical_accuracy: 0.2016 - val_loss: 2.2002 - val_categorical_accuracy: 0.2136
Epoch 2/10
 2742/25000 [==>...........................] - ETA: 5631s - loss: 2.2235 - categorical_accuracy: 0.2060

In [15]:
model.save('model_state1.h5')

In [7]:
from keras.models import load_model
model = load_model('model_state1.h5')

## Showing the training summary

In [ ]:

import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training losses')
plt.plot(epochs, val_loss, 'b', label='Validation losses')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# Fine tuning

In [ ]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
        
model.compile(
    optimizer=optimizers.RMSprop(lr=1e-4), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy]
)

history = model.fit_generator(train_generator, 
                              steps_per_epoch=12600, 
                              epochs=10, 
                              validation_data=test_generator, 
                              validation_steps=900)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training losses')
plt.plot(epochs, val_loss, 'b', label='Validation losses')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model.save('model.h5')